In [ ]:
#default_exp prepare_data

# Etude des sujets des amendements PLFSS

## Préparation des données

In [ ]:
#!python -m pip install --upgrade pip

In [ ]:
#!pip install -q --upgrade pandas  spacy  more_itertools

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!python -m spacy download fr_core_news_sm > /dev/null

In [ ]:
# Importing modules
import pandas as pd
#amdt = pd.read_csv('../../../donnees-extraites-assemblee/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt = pd.read_csv('https://github.com/leximpact/donnees-extraites-assemblee/raw/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt.head(1)

,texteLegislatifUid,uid,avantAApres,dispositif,exposeSommaire
0,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000001,A,"I. – Après l’alinéa 13, insérer l’alinéa suiv...",La mise en place d’un accord d’intéressement d...


In [ ]:
# FOR DEBUG
# Keep a random sample
amdt = amdt.sample(n=100,replace=False, random_state=1)

In [ ]:
len(amdt)

100

In [ ]:
#amdt = amdt.drop(columns=['texteLegislatifUid', 'avantAApres'], axis=1)

In [ ]:
amdt['texte'] = amdt['dispositif'] + amdt['exposeSommaire']

## Nettoyage

### Retrait des mots de liaison (_stopword_) et de la ponctuation

In [ ]:
test = "Bonjour Madame Marianne et Monsieur Coq,\n Comment allez-vous ?"
print(test)

Bonjour Madame Marianne et Monsieur Coq,
 Comment allez-vous ?


In [ ]:
import spacy
#from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
nlp = spacy.load("fr_core_news_sm")

In [ ]:

def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    #print(doc[4].pos_)
    # Retire la ponctuation et les retours à la ligne
    return [X.text for X in doc if X.is_punct is False | X.is_space is False | X.is_stop is False ]
test_token = return_token(test)
test_token

['Bonjour', 'Madame', 'Marianne', 'Monsieur', 'Coq', 'allez', '-vous']

In [ ]:
data_words = []

def str_token(text):
    global data_words
    tokens = return_token(text)
    data_words += tokens
    return " ".join(tokens)

amdt['txt_sans_stopword'] = amdt['texte'].apply(str_token)

In [ ]:
len(data_words)

10756

In [ ]:
import pickle
pickle.dump( data_words, open( "./data/amdt_data_words.pickle", "wb" ) )
amdt['txt_sans_stopword'].to_csv('./data/amdt_sans_stopword.csv.gz', compression='gzip')

# WIP : Les entites les plus cités
Spacy permet d'extraire les nom de personne, de lieu, les dates, cela pourrait servir...

In [ ]:
doc = nlp(test)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Bonjour Madame Marianne 0 23 PER
Monsieur Coq 27 39 PER
